# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')
api_key = "ollama"

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
MODEL = "llama3.2"
# openai = OpenAI()
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = (
    "You are provided with a list of links found on a webpage."
    "You are able to decide which of the links would be most relevant to include in a brochure about the company,"
    "such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
)
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
  "links": [
    {"type": "about page", "url": "https://full.url/goes/here/about"},
    {"type": "careers page": "url": "https://another.full.url/careers"}
  ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.You are able to decide which of the links would be most relevant to include in a brochure about the company,such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
  "links": [
    {"type": "about page", "url": "https://full.url/goes/here/about"},
    {"type": "careers page": "url": "https://another.full.url/careers"}
  ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += (
        "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format."
        "Do not include Terms of Service, Privacy, email links.\n"
    )
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format.Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/hexgrad/Kokoro-82M',
 '/openbmb/MiniCPM-o-2_6',
 '/microsoft/phi-4',
 '/NovaSky-AI/Sky-T1-32B-Preview',
 '/deepseek-ai/DeepSeek-V3',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/FaceOnLive/Face-Search-Online',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/stabilityai/stable-point-aware-3d',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/NovaSky-AI/Sky-T1_data_17k',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/DAMO-NLP-SG/multimodal_textbook',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transforme

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://ui.endpoints.huggingface.co'},
  {'type': 'learn more about the company',
   'url': 'https://huggingface.co/blog'},
  {'type': 'docs and research', 'url': 'https://docs.huggingface.co'},
  {'type': 'company page on LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'GitHub organization', 'url': 'https://github.com/huggingface'},
  {'type': 'Join the community on Discord',
   'url': 'https://join.discord.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'Company home page', 'url': 'https://huggingface.co'}, {'type': 'About page', 'url': 'https://huggingface.co/brand'}, {'type': 'Careers/Jobs page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog page', 'url': 'https://discuss.huggingface.co/threads/'}, {'type': 'News/Status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn Company Profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
hexgrad/Kokoro-82M
Updated
2 days ago
•
20.6k

In [31]:
# system_prompt = (
#     "You are an assistant that analyzes the contents of several relevant pages from a company website"
#     "and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."
#     "Include details of company culture, customers and careers/jobs if you have the information."
# )

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = (
    "You are an assistant that analyzes the contents of several relevant pages from a company website"
    "and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown."
    "Include details of company culture, customers and careers/jobs if you have the information."
)

In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += "Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'Company/About page', 'url': 'https://huggingface.co'}, {'type': 'Home page', 'url': 'https://huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nhexgrad/Kokoro-82M\nUpdated\n2 days ago\n•\n20.6k\n•\n1.85k\nopenbmb/MiniCPM-o-2_6\nUpdated\nabout 15 hours ago\n•\n7.05k\n•\n530\nmicrosoft/phi-4\nUpdated\n9 days ago\n•\n100k\n•\n1.41k\nNovaSky-AI/Sky-T1-32B-Preview\nUpdated\n5 days ago\n•\n4.89k\n•\n444\ndeepseek-ai/DeepSeek-V3\nUpdated\n19 days ago\n•\n142k\n•\n2k\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.04k\n❤️\nKokoro TTS\nNow in 5 languages!\nRunning\non\nZero\n3.09

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [28]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co/brand'}, {'type': 'Jobs/Careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


# Hugging Face Brochure

[Cover Page]

Hugging Face is the leading platform for the machine learning community, building the future of Artificial Intelligence.

[About Us]
-----------------------------

At Hugging Face, we believe that AI has the power to transform the world. We're on a mission to empower the next generation of AI researchers and developers by providing a collaborative platform where they can create, discover, and apply ML models.

Our vision is to make machine learning accessible, scalable, and versatile for everyone.

[Mission]
----------

*   Enable the machine learning community to collaborate on models, datasets, and applications
*   Provide a seamless integration of various open-source stacks, frameworks, and tools
*   Foster innovation through our research initiatives, workshops, and online courses
*   Empower businesses and researchers alike with cutting-edge AI solutions

[Key Strengths]

Hugging Face has:

*   400k+ pre-trained models available
*   A large community of 50k+ organizations worldwide
*   Over 150k+ applications in our catalog
*   Extensive documentation, tutorials, and support

[Customer Base]
-----------------

Many of the world's top companies trust Hugging Face to help them develop innovative AI solutions:

*   *Meta*: AI2 Enterprise Partner for various AI tools.
*   *Meta*'s *AI Technology Lab: Collaborates extensively with TheHuggingFaceProject team*
*   *Amazon Web Services*: Uses our open-source model and toolkit integration into the AWS cloud platform services to help customers better build data applications and improve their ability to integrate machine learning models
*   *Microsoft*: Features various AI Tools that are a part of HuggingFace ecosystem and integrated tools in Windows.
*   *AI for social Impact*: Google, company's AI Platform provides an interface into various popular machine learning frameworks like TensorFlow.

[Careers]
---------

**At Hugging Face: We**

We're excited to collaborate with the best minds in the field, providing opportunities for growth:

*   Join our vibrant community
*   Explore the latest developments 
*   Be part of shaping AI's next great breakthroughs.

**What We Offer**

Career Opportunities at HuggingFace:

*   *Research & Development*: Help us build state-of-the-art infrastructure, models, and applications.
*   *Product Engineer*: Work on our various initiatives: development framework, APIs, and tools for integrating multiple components into products.

[Join the Conversation]
---------------------

Follow us on social media to stay up-to-date with our latest updates:

LinkedIn
Twitter
Discord

Get in touch with our team or learn more about open-source projects:

GitHub Blog Forum Jobs Press

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [29]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [30]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'Company page', 'url': 'https://huggingface.co/brand'}]}


Hugging Face Brochure
========================

Welcome to Hugging Face, the platform where the machine learning community collaborates on models, datasets, and applications.

**About Us**
---------------

At Hugging Face, we are building the foundation of ML tooling with the community. Our mission is to empower innovators across industries by making it easy to develop, deploy, and scale AI-powered solutions.

**What We Do**
-----------------

* Provide a collaboration platform for the machine learning community
* Offer a vast library of models and datasets for various tasks, including text, image, video, audio, and 3D generation
* Enable users to host and collaborate on unlimited public models, datasets, and applications
* Offer paid Compute and Enterprise solutions for secure and scalable deployment

**Key Features**

* **Transformers**: State-of-the-art ML for Pytorch, TensorFlow, and JAX
* **Diffusers**: State-of-the-art diffusion models for image and audio generation in PyTorch
* **Safetensors**: Simple, safe way to store and distribute neural networks weights safely and quickly
* **Hub Python Library**: Client library for the HF Hub: manage repositories from your Python runtime

**Community**
-------------

Hugging Face has welcomed over 50,000 organizations across various industries, including AI2, Enterprise, non-profit, Meta, Amazon Web Services, Google, Intel, Microsoft, Grammarly, Writer, and more.

**Open Source**
----------------

We are committed to building the foundation of ML tooling with the community. Our open-source projects include:

* **Transformers**: 137,720 downloads
* **Diffusers**: 27,151 downloads
* **Safetensors**: 3,011 downloads
* **Hub Python Library**: 2,231 downloads

**Join Us**
-------------

Ready to become a part of the Hugging Face community? Explore our [GitHub repository](https://github.com/huggingface) and join us on [Twitter](https://twitter.com/HuggingFace).

Get started with Hugging Face today and unlock your ML potential!

In [33]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'About page', 'url': 'https://huggingface.co/brand'}, {'type': 'Company page', 'url': 'https://huggingface.co/'}, {'type': 'Careers/Jobs page', 'url': 'https://ui.endpoints.huggingface.co/chat'}, {'type': 'Blog', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter handle', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face!
===========================

Your partner in building a better, more sustainable future through AI!

[Image: A collage of various faces from different organizations, with a subtle background of neural networks]

## About Us
We believe that artificial intelligence should be for everyone. That's why we're on a mission to make machine learning accessible, affordable, and – most importantly – fun.

**Our Story**
----------

In 2016, our founders set out to solve the challenge of machine learning model deployments. What started as a simple solution soon grew into something much bigger: a community-driven platform where anyone can build, share, and collaborate on ML models.

**What We Do**
-------------

We are a collaboration platform for the machine learning community. Our platform allows users to:

* Share and discover open-source models
* Experiment with different models and datasets
* Build and deploy their own models
* Learn from others in our vibrant community

**Our Open Source Creations**
---------------------------

Some of our most notable projects include:

* **Transformers**: A state-of-the-art library for natural language processing
* **Diffusers**: A suite of tools for image and audio generation
* **PEFT**: A parameter-efficient finetuning method for large models

**Community Spotlight**
--------------------

Meet some of the amazing organizations that trust Hugging Face:

* Meta
* Amazon Web Services
* Google
* Intel
* Microsoft
* Grammarly
* Writer (Enterprise)

[Image: A map pin with these logos]

## Our Culture
We're a team of passionate machine learning enthusiasts who believe in open-world collaboration and shared AI prosperity. Here are some key values that make us tick:

* **Community First**: We put the needs of our users at the forefront of everything we do.
* **Inclusive Design**: We strive to create products and experiences that are accessible to everyone, regardless of background or skill level.
* **Sustainable AI**: We're committed to building an open-source foundation for machine learning tooling.

**Join Our Crew!**
-----------------

Ready to join the thousands of folks who have already started their Hugging Face journey?

* Check out our [jobs page](#jobs)
* Explore our [open source repository](#open-source-projects)

[Image: A hand holding a puzzle piece, with the words "Build, Collaborate, Repeat"]

## Let's Build the Future Together!
---------------------------------

Join us on this exciting adventure into the world of machine learning. We can't wait to see what you'll create!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>